In [7]:
import pandas as pd

# Загружаем данные с правильным разделителем — точка с запятой
df = pd.read_csv("skempi_v2.csv", sep=';')

# Смотрим первые строки таблицы
df.head()

,#Pdb,Mutation(s)_PDB,Mutation(s)_cleaned,iMutation_Location(s),Hold_out_type,Hold_out_proteins,Affinity_mut (M),Affinity_mut_parsed,Affinity_wt (M),Affinity_wt_parsed,...,koff_mut_parsed,koff_wt (s^(-1)),koff_wt_parsed,dH_mut (kcal mol^(-1)),dH_wt (kcal mol^(-1)),dS_mut (cal mol^(-1) K^(-1)),dS_wt (cal mol^(-1) K^(-1)),Notes,Method,SKEMPI version
0,1CSE_E_I,LI45G,LI38G,COR,Pr/PI,Pr/PI,5.26E-11,5.260000e-11,1.12E-12,1.120000e-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IASP,1
1,1CSE_E_I,LI45S,LI38S,COR,Pr/PI,Pr/PI,8.33E-12,8.330000e-12,1.12E-12,1.120000e-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IASP,1
2,1CSE_E_I,LI45P,LI38P,COR,Pr/PI,Pr/PI,1.02E-07,1.020000e-07,1.12E-12,1.120000e-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IASP,1
3,1CSE_E_I,LI45I,LI38I,COR,Pr/PI,Pr/PI,1.72E-10,1.720000e-10,1.12E-12,1.120000e-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IASP,1
4,1CSE_E_I,LI45D,LI38D,COR,Pr/PI,Pr/PI,1.92E-09,1.920000e-09,1.12E-12,1.120000e-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IASP,1


In [8]:
# Считаем ddG вручную
df['ddG'] = df['Affinity_mut_parsed'] - df['Affinity_wt_parsed']

# Создаём бинарную метку: 1 если ddG > 0, иначе 0
df['sign_ddG'] = df['ddG'].apply(lambda x: 1 if x > 0 else 0)

# Покажем результат
df[['Affinity_mut_parsed', 'Affinity_wt_parsed', 'ddG', 'sign_ddG']].head()

,Affinity_mut_parsed,Affinity_wt_parsed,ddG,sign_ddG
0,5.260000e-11,1.120000e-12,5.148000e-11,1
1,8.330000e-12,1.120000e-12,7.210000e-12,1
2,1.020000e-07,1.120000e-12,1.019989e-07,1
3,1.720000e-10,1.120000e-12,1.708800e-10,1
4,1.920000e-09,1.120000e-12,1.918880e-09,1


In [10]:
# Удалим строки, где есть пустые значения (NaN)
df_clean = df.dropna(subset=['Affinity_mut_parsed', 'Affinity_wt_parsed', 'ddG'])

# Обновим X и y
X = df_clean[['Affinity_mut_parsed', 'Affinity_wt_parsed']]
y = df_clean['sign_ddG']

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Точность модели:", accuracy)

Точность модели: 0.7580882352941176
